In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime
from urllib.parse import urlparse

In [27]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

 
driver.get("https://www.bbc.com/news")
print(driver.title)

Home - BBC News


In [28]:
news_urls=[]
def get_urls():
 elements_for_url=driver.find_elements(By.CLASS_NAME, 'gs-c-promo-heading')
 for element in elements_for_url:
        news_urls.append(element.get_attribute('href'))
 print(len(elements_for_url))
get_urls()
print(len(news_urls))#list with all the urls to the articels. should be equally long to elements_for_url
print(news_urls[64])

75
75
https://www.bbc.com/sport/cricket/67644896


In [29]:
def remove_numbers(category):
    # Find the position of the first digit in the category
    digit_index = next((index for index, char in enumerate(category) if char.isdigit()), None)

    # Remove everything after the first digit
    category_without_number = category[:digit_index] if digit_index is not None else category

    # Remove trailing hyphen
    if category_without_number.endswith('-'):
        category_without_number = category_without_number[:-1]


    index_of_hyphen=category_without_number.rfind('-')#search starting from the right
    if index_of_hyphen!= -1:#if there was a hyphen
       result_string=category_without_number[index_of_hyphen+1:]#cut everything to the left
       return result_string
    else:
      return category_without_number


def get_topic(url):
 try:
  parsed_url = urlparse(url)
  path_segments = parsed_url.path.split('/')

# Find the segment after "news"
  category_segment_index = path_segments.index("news") + 1
  category = path_segments[category_segment_index]

  category_without_number = remove_numbers(category)

  return category_without_number
 except:

  try:
   parsed_url = urlparse(url)
   path_segments = parsed_url.path.split('/')
   category_segment_index = path_segments.index("sport") + 1
   category = path_segments[category_segment_index]


   category_without_number = remove_numbers(category)

   return category_without_number
 
  except:
   
   return "no info"


In [30]:
#here we extract all the text contents from the articles. This takes approximately 2 minutes with a total of 76 articels today
def extract_contents(text_contents):
  try:
   article = driver.find_element(By.TAG_NAME, 'article')
   text_contents.append(article.text)
  except: #exception if no article element is there. doesn´t happen often, but has to be handled
   article = driver.find_element(By.TAG_NAME, 'body')
   text_contents.append(article.text)
   print("article not found") #in this test, 3 of 73 articels had no article element. 
#extract_contents()
#print(len(text_contents)) 

In [31]:

#here are a few exceptions, because the bbc articles have different html structures
def get_headers(headers):
  try:
   header=driver.find_element(By.ID, 'main-heading')
   headers.append(header.text)
  except:
   try:
     header = driver.find_element(By.CLASS_NAME, 'qa-story-headline')
     headers.append(header.text)
   except:
    try:
     header = driver.find_element(By.CLASS_NAME, 'article-headline__text')
     headers.append(header.text)
    except:
     header=None
     headers.append(header)
     print('no header')
#get_headers()




In [32]:
def get_timestamps(time):
  try:
   time_element=driver.find_element(By.TAG_NAME, 'time')
   date=(time_element.get_attribute('datetime'))
   time.append(date.split(".")[0])#ignore the error, it works when it´s accessed by the main function
  except:
    date=None
    time.append(date)
    print('no date')

In [33]:

def get_Image(imageURL,ImageDesc):
   try:
    image_element=driver.find_element(By.TAG_NAME, 'img')
    url=(image_element.get_attribute('src'))
    desc=(image_element.get_attribute('alt'))
    imageURL.append(url)
    ImageDesc.append(desc)
   except:
    url=None
    desc=None
    imageURL.append(url)
    ImageDesc.append(desc)
    print('no image')

In [34]:
#main method to extract all contents except the authors(might follow later). Runs way faster than the different smaller methods before
text_contents=[]
time=[]
headers=[]
imageURL=[]
imageDesc=[]
topic=[]
def extract_all():
    for i in range(len(news_urls)):
       driver.get(news_urls[i]) #we go to the specific article by using the url
       parsed_url = urlparse(news_urls[i])
       extract_contents(text_contents)
       get_timestamps(time)
       get_headers(headers)
       get_Image(imageURL,imageDesc)
       topic_result = get_topic(news_urls[i])
       topic.append(topic_result)
extract_all()
print(text_contents[10])
print(time[10])
print(headers[10])
print(imageURL[10])
print(imageDesc[10])
print(topic[10])


no date
no header
article not found
no date
article not found
no date
no header
no date
no date
no date
no date
no date
no date
no date
Unesco: Italian opera singing gets cultural heritage status
Published
5 hours ago
Share
IMAGE SOURCE,
GETTY IMAGES
Image caption,
Late Italian tenor Luciano Pavarotti was a great exponent of opera
By Nadia Ragozhina
BBC News
Italian opera singing has been added to the United Nations' cultural heritage list.
Unesco, the UN culture agency, announced it was including the four-centuries-old art, mixing costume, drama and music, under its category of Intangible Cultural Heritage.
Italian opera developed in the late 1500s and early 1600s in Florence, at the court of the Medici family.
It typically involves intricate plots, elaborate staging and virtuoso singing.
Opera singing joins other famed Italian practices, such as pizza-making and the art of scaling the Alpine peaks.
"This is an official confirmation of what we already knew: opera singing is a world ex

In [35]:
for item in headers:
  print(item)
print(len(headers))
counter=0
for i in range(len(headers)):
    if headers[i]==None:
      counter=counter+1
print(counter)
for item in news_urls:
  print(item)

None
Russia luring migrants from Finnish border for war in Ukraine
Benjamin Zephaniah: Writer, poet and Peaky Blinders actor dies aged 65
Russia hacking: 'FSB in years-long cyber attacks on UK', says government
Bringing broken survivors of the Israel Nova festival attack back from darkness
UNLV shooting: Former college professor kills three on Las Vegas campus
Watch: Video shows moment of huge explosion in Seychelles
Denmark passes law to ban Quran burnings
Ukraine aid from US in doubt after failed Senate vote
Nikki Haley under attack and more Republican debate takeaways
Unesco: Italian opera singing gets cultural heritage status
Nikki Haley under attack and more Republican debate takeaways
Unesco: Italian opera singing gets cultural heritage status
Venezuela accuses opposition politicians of treason over Essequibo
Get behind my Rwanda asylum plan, Rishi Sunak tells Tories
Russia school shooting: Teenage girl kills classmate and herself
BBC reporter in Gaza: People here are traumatised

In [36]:
def get_authors(article):
    authors = []
    lines = article.split('\n')
    for line in lines[:13]:  # Loop through lines from lines[0] to lines[10]
        if "By" in line:
            authors.append(line.replace('By', ''))
    return authors

all_authors = []
def get_all_authors():
    for article in text_contents:
        authors = get_authors(article)
        if not authors:  # Check if authors is an empty list
            all_authors.append(None)
        else:
            all_authors.extend(authors)

# Print the authors
for author in all_authors:
    print(author)


get_all_authors()
print(all_authors[10])
print(len(all_authors))
print(all_authors)

counter=0
for i in range(len(all_authors)):
    if all_authors[i]==None:
      counter=counter+1
print(counter)

#<p class="lx-commentary__meta-reporter gel-long-primer" data-reactid=".18n36quudgy.0.2.0.0.1.0.1.0.$lx-tabs0.0.$lx-commentary.$lx-commentary.0.2.0.0">  Edited by Nadia Ragozhina and Samuel Horti</p>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By Jeremy Bowen</div>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By David Gritten</div>

 Nadia Ragozhina
77
[None, ' Oleg Boldyrev', ' Steven McIntosh', ' Gordon Corera', ' Anna Foster', ' Nadine Yousif', None, ' Jaroslav Lukiv', ' Bernd Debusmann Jr', ' Kayla Epstein', ' Nadia Ragozhina', ' Kayla Epstein', ' Nadia Ragozhina', ' Vanessa Buschschlüter', ' Joshua Nevett', None, ' Adnan El-Bursh in Khan Younis, Gaza', ' Francesca Gillett', ' Frank Gardner', None, ' Lucy Williamson', ' Steve Rosenberg', None, None, ' Kelly Ng & BBC Chinese', ' Jana Tauschinski, Becky Dale and Esme Stallard', ' Oliver Englehart, Mark Casebow & Laura Gaynor', ' Hanna Samosir & Astudestra Ajengrastri', ' Jessica Parker', None, None, None, None, None, ' Kelly Ng', ' Mariko Oi', None, ' Mark Savage', ' Yasmin Rufo', ' Shiona McCallum and Zoe Kleinman', ' Paul Seddon', ' Oleg Boldyrev', ' Nadine Yousif', ' Steven McIntosh', ' Mariko Oi', ' Kayla Epstein', ' Gordon Corera', None, ' Patrick Muirhead', ' Gloria Aradi & Priya Sippy', ' Steve Rosenberg', ' Sadiqur Rahman', ' Katie Bishop', ' Alex Ehrenr

In [37]:
#dictionary with all infos, ordered after attributes
from datetime import datetime
import pytz

germany_timezone = pytz.timezone("Europe/Berlin")


articles_info={}
for i in range(len(news_urls)):
 article_info_i={ 
  "headline": headers[i],
  "url": news_urls[i],
  "date":time[i],
  "authors":[all_authors[i]],
  "ImageURL":imageURL[i],
  "ImageDescription":imageDesc[i],
  "scrapingTimeStamp":(datetime.now(germany_timezone).isoformat()).split(".")[0],
  "source":"bbc",
  "topics":topic[i],
  "contents":text_contents[i]
 }
 articles_info[i]=article_info_i
print(articles_info[50])
print(topic[65])
#URLId, Headline, Contents, Authors, UploadDate, ReadTime, ImageURL, ImageDescription



{'headline': 'Karin Kneissl, the Austrian ex-minister who moved to Russia', 'url': 'https://www.bbc.com/news/world-europe-67624834', 'date': '2023-12-07T06:01:28', 'authors': [' Steve Rosenberg'], 'ImageURL': 'https://ichef.bbci.co.uk/news/976/cpsprodpb/5C14/production/_131927532_kneisslwedding2018-rolandschlager-poolviareuters.jpg', 'ImageDescription': 'Putin dances with Kneissl at her wedding, August 2018', 'scrapingTimeStamp': '2023-12-07T18:23:39', 'source': 'bbc', 'topics': 'europe', 'contents': 'Karin Kneissl, the Austrian ex-minister who moved to Russia\nPublished\n11 hours ago\nShare\nRelated Topics\nWar in Ukraine\nIMAGE SOURCE,\nREUTERS\nImage caption,\nVladimir Putin danced with Ms Kneissl at her wedding in 2018, and brought her a bouquet of flowers\nBy Steve Rosenberg\nRussia Editor, St Petersburg\nIt\'s all rather surreal.\nI\'m sitting in a 19th Century St Petersburg mansion. Lying on the floor by my chair is Winston Churchill. He is fast asleep… and he is snoring.\n"Mayb

In [38]:
import psycopg2

In [39]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [40]:
for article in articles_info.values():
    try:
        cursor.execute('''INSERT INTO Articles(URLId, Headline, Content, Authors, UploadTimestamp, ImageURL, ImageDescription,scrapingTimeStamp, source, topic) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''', 
                       (article["url"], article["headline"], article["contents"], article["authors"], article["date"], article["ImageURL"], article["ImageDescription"], article["scrapingTimeStamp"], article["source"], article["topics"])
                       )
    except psycopg2.IntegrityError as e:
        if "duplicate key value violates unique constraint" in str(e):
            print(f"Article with URLId {article['url']} already exists in the database.")






Article with URLId https://www.bbc.com/news/world-us-canada-67645334 already exists in the database.
Article with URLId https://www.bbc.com/news/world-europe-67647468 already exists in the database.
Article with URLId https://www.bbc.com/news/world-67650491 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67644334 already exists in the database.
Article with URLId https://www.bbc.com/news/world-europe-67647379 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67642512 already exists in the database.
Article with URLId https://www.bbc.com/news/entertainment-arts-67646607 already exists in the database.
Article with URLId https://www.bbc.com/news/business-67644926 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67645334 already exists in the database.
Article with URLId https://www.bbc.com/news/uk-politics-67647548 already exists in the database.
Article with